In [166]:
import lmdb
from os.path import join
import os
import caffe
import numpy as np
import hashlib
from collections import OrderedDict
import re

In [167]:
db_path_train = join('/home/aachenhang/dataset/cc/data', 'ShanghaiTech_BDB', 'Ratio0.56', 'train-count')
db_path_test = join('/home/aachenhang/dataset/cc/data', 'ShanghaiTech_BDB', 'Ratio0.56', 'test-count')

In [168]:
def is_image(im):
    return ('.jpg' in im) or ('.JPG' in im) or ('.PNG' in im) or ('.png' in im)
data = join('/home/aachenhang/dataset', 'frames_ShanghaiTech_B')

In [169]:
img_list = [img for img in os.listdir(data) if is_image(img)]
assert (len(img_list) != 0)
img_list = sorted(img_list)

In [170]:
NTrain = 400
NTest = 316
tol = 3

train_list = img_list[:NTrain]
assert (len(train_list) != 0)

test_list = img_list[NTrain:NTrain + NTest]
assert (len(test_list) != 0)


train_cnt = np.array([])
minCount = 0
scale = 1
with open(join('/home/aachenhang/dataset/cc', 'data','ShanghaiTech_BDB', 'Ratio0.56','db.info')) as db_info:
    l = db_info.readlines()[1].split(',')  
    minCount = int(re.sub("[^0-9]", "", l[2]))
    scale = float(re.sub("[^0-9\.]", "", l[-1]))
print "minCount : ", minCount
print "scale : ", scale



minCount :  2
scale :  5.78


In [171]:
with lmdb.open(db_path_train, map_size=int(1e12)) as db_train:
    in_txn = db_train.begin(write=False)
    IDX_FMT = '{:0>%d' % 10 + 'd}'
    datum = caffe.proto.caffe_pb2.Datum()
    for i, img in enumerate(train_list):
        datum.ParseFromString(in_txn.get(IDX_FMT.format(i)))
        arr = caffe.io.datum_to_array(datum)
        pred = arr[:,0,0].argsort()[-1] + minCount
        pred = scale * pred
        label = int(re.sub("[^0-9]", "", img.split('n')[-1]))
        if(abs(pred - label) > tol):
            print "found error data!"
            print "index : ", i
            print "img name : ", img
            print "pred : ", pred
            print "label : ", label
            raise Exception("Found error data!")
    print "db_train pass"
        

found error data!
index :  0
img name :  seq_0001n234.jpg
pred :  231.2
label :  234


Exception: Found error data!

In [ ]:
with lmdb.open(db_path_test, map_size=int(1e12)) as db_test:
    in_txn = db_test.begin(write=False)
    IDX_FMT = '{:0>%d' % 10 + 'd}'
    datum = caffe.proto.caffe_pb2.Datum()
    for i, img in enumerate(test_list):
        datum.ParseFromString(in_txn.get(IDX_FMT.format(i)))
        arr = caffe.io.datum_to_array(datum)
        pred = arr[:,0,0].argsort()[-1] + minCount
        pred = scale * pred
        label = int(re.sub("[^0-9]", "", img.split('n')[-1]))
        if(abs(pred - label) > tol):
            print "found error data!"
            print "index : ", i
            print "img name : ", img
            print "pred : ", pred
            print "label : ", label
            raise Exception("Found error data!")
    print "db_test pass"
        